In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpcaufz449".


In [5]:
%%writefile dtw_naive_kernel.cu
#include <stdio.h>
#include <climits>

template <typename index_t, typename value_t> __global__
void DTW_naive_kernel(
  value_t *Query,
  value_t *Subject,
  value_t *Dist,
  value_t *Cache,
  index_t num_entries,
  index_t num_features) {
    const index_t thid = blockDim.x * blockIdx.x + threadIdx.x;
    const index_t lane = num_features + 1;
    const index_t base = thid * num_features;

    if (thid < num_entries) {
      value_t *penalty = Cache + thid * 2 * lane;
      penalty[0] = 0;

      for (index_t index = 0; index < lane; index++)
        penalty[index + 1] = INFINITY;

      for (index_t row = 1; row < lane; row++) {
        const value_t q_value = Query[row - 1];
        const index_t target_row = row & 1;
        const index_t source_row = !target_row;
        if (row == 2) penalty[target_row * lane] = INFINITY;
        for (index_t col = 1; col < lane; col++) {
          const index_t src_off = source_row * lane;
          const index_t trg_off = target_row * lane;
          const value_t diag = penalty[src_off + col - 1];
          const value_t abve = penalty[src_off + col - 0];
          const value_t left = penalty[trg_off + col - 1];
          const value_t s_value = Subject[base + col - 1];
          const value_t residue = q_value - s_value;
          penalty[target_row * lane + col] = residue * residue + std::min(diag, std::min(abve, left));
        }
      }
      const index_t last_row = num_features & 1;
      Dist[thid] = penalty[last_row * lane + num_features];
    }
  }

int main() {}

Overwriting dtw_naive_kernel.cu


In [6]:
!nvcc -arch=sm_75 -o dtw_naive_kernel dtw_naive_kernel.cu